# Setup

I've created a free AI Search instance for this lab.

The sample data is about the [2024 UEFA Euro tournament](data/2024_UEFA_Euro.txt), indexed in Azure AI Search.

In [21]:
import os

from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

OPENAI_ENDPOINT = os.getenv("OPENAI_ENDPOINT")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
MODEL_DEPLOYMENT = os.getenv("MODEL_DEPLOYMENT")
API_VERSION = os.getenv("API_VERSION")

AI_SEARCH_ENDPOINT = os.getenv("AI_SEARCH_ENDPOINT")
AI_SEARCH_KEY = os.getenv("AI_SEARCH_KEY")
AI_SEARCH_INDEX_NAME = os.getenv("AI_SEARCH_INDEX_NAME")
EMBEDDING_MODEL_DEPLOYMENT = os.getenv("EMBEDDING_MODEL_DEPLOYMENT")

In [22]:
chat_client = AzureOpenAI(
    api_version=API_VERSION,
    azure_endpoint=OPENAI_ENDPOINT,
    api_key=OPENAI_API_KEY,
)

# Usage

In [37]:
prompt = [
    {"role": "system", "content": "You are a helpful AI assistant."},
    {"role": "user", "content": "Who are the players that score in the 2024 Euro final? And what are the players that won the personal prizes?"}
]

## Without RAG

In [50]:
response = chat_client.chat.completions.create(
    model=MODEL_DEPLOYMENT,
    messages=prompt,
    temperature=0,
)

print(response.choices[0].message.content)

I don't have real-time data access, including the results of the 2024 Euro final or the personal prizes awarded. For the most accurate and up-to-date information, please refer to official sources such as UEFA's website, sports news outlets, or the official Euro 2024 tournament website.


## With RAG (keyword-based search)

In [51]:
# Additional parameters to apply RAG pattern using the AI Search index
rag_params = {
    "data_sources": [
        {
            "type": "azure_search",
            "parameters": {
                "endpoint": AI_SEARCH_ENDPOINT,
                "index_name": AI_SEARCH_INDEX_NAME,
                "authentication": {
                    "type": "api_key",
                    "key": AI_SEARCH_KEY,
                }
            }
        }
    ],
}

response = chat_client.chat.completions.create(
    model=MODEL_DEPLOYMENT,
    messages=prompt,
    temperature=0,
    extra_body=rag_params
)

print(response.choices[0].message.content)

The players that scored in the 2024 Euro final were Nico Williams, who scored for Spain, and Cole Palmer, who scored for England [doc1][doc2]. The players that won the personal prizes were Rodri, who won the Player of the Tournament, and Lamine Yamal, who won the Young Player of the Tournament [doc1].


## With RAG (vector-based search)

In [49]:
rag_params = {
    "data_sources": [
        {
            "type": "azure_search",
            "parameters": {
                "endpoint": AI_SEARCH_ENDPOINT,
                "index_name": AI_SEARCH_INDEX_NAME,
                "authentication": {
                    "type": "api_key",
                    "key": AI_SEARCH_KEY,
                },
                # Params for vector-based query
                "query_type": "vector",
                "embedding_dependency": {
                    "type": "deployment_name",
                    "deployment_name": EMBEDDING_MODEL_DEPLOYMENT,
                },
            }
        }
    ],
}

response = chat_client.chat.completions.create(
    model=MODEL_DEPLOYMENT,
    messages=prompt,
    temperature=0,
    extra_body=rag_params
)

print(response.choices[0].message.content)

In the 2024 UEFA Euro final, the players who scored were Lamine Yamal for Spain and Cole Palmer for England [doc1][doc2]. The players who won the personal prizes were Rodri for the Player of the Tournament and Lamine Yamal for the Young Player of the Tournament [doc1].
